# CS 437/5317 Deep Learning - Tutorial 6

In [20]:
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout, Input

## Pretrained models available in Keras

https://keras.io/applications

## Implementation of Transfer Learning

In [15]:
num_classes = 10
input_shape = (224,224,3)

### Approach 1

In [16]:
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False)

for layer in vgg_conv.layers:
    layer.trainable = False

In [17]:
t = Flatten()(vgg_conv.layers[-1].output)
t = Dense(1024, activation='relu')(t)
t = Dropout(0.5)(t)
output = Dense(num_classes, activation='softmax')(t)

model = Model(vgg_conv.layers[0].input, output)

In [18]:
model.summary()
model.compile(optimizer='adam',
              loss='mse')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

### Approach 2

In [25]:
#Load the VGG model
vgg_conv = VGG16(weights=None, include_top=False, input_shape=input_shape)

for layer in vgg_conv.layers:
    layer.trainable = False

In [26]:
im = Input(shape=input_shape)
l = vgg_conv(im)
l = Flatten()(l)
l = Dense(1024, activation='relu')(l)
l = Dropout(0.5)(l)
output = Dense(num_classes, activation='softmax')(l)

model = Model(im, output)

In [27]:
model.summary()
model.compile(optimizer='adam',
              loss='mse')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_7 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 1024)              25691136  
_________________________________________________________________
dropout_6 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                10250     
Total params: 40,416,074
Trainable params: 25,701,386
Non-trainable params: 14,714,688
_______________________________________________________

## Autoencoder implementation using Functional API

In [55]:
# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats (28x28)

In [56]:
# this is our input placeholder
input_img = Input(shape=(784,), name='input_image')
# "encoded" is the encoded representation of the input
encoded = Dense(200, activation='relu', name='encoder_layer1')(input_img)
encoded = Dense(encoding_dim, activation='relu', name='encoder_layer1')(encoded)
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded, name='encoder_model')

In [57]:
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_image (InputLayer)     (None, 784)               0         
_________________________________________________________________
encoder_layer1 (Dense)       (None, 32)                25120     
Total params: 25,120
Trainable params: 25,120
Non-trainable params: 0
_________________________________________________________________


In [58]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,), name='encoded_input')
# "decoded_output" is the lossy reconstruction of the input
decoded_output = Dense(200, activation='sigmoid', name='decoder_layer1')(encoded_input)
decoded_output = Dense(784, activation='sigmoid', name='decoder_layer1')(decoded_output)
# create the decoder model
decoder = Model(encoded_input, decoded_output, name='decoder_model')

In [59]:
decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoded_input (InputLayer)   (None, 32)                0         
_________________________________________________________________
decoder_layer1 (Dense)       (None, 784)               25872     
Total params: 25,872
Trainable params: 25,872
Non-trainable params: 0
_________________________________________________________________


In [62]:
# this model maps an input to its reconstruction
input_img = Input(shape=(784,), name='input_image')
encoded = encoder(input_img)
decoded = decoder(encoded)

autoencoder = Model(input_img, decoded, name='autoencoder_model')

In [63]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_image (InputLayer)     (None, 784)               0         
_________________________________________________________________
encoder_model (Model)        (None, 32)                25120     
_________________________________________________________________
decoder_model (Model)        (None, 784)               25872     
Total params: 50,992
Trainable params: 50,992
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# encode and decode some digits
encoded_imgs = encoder.predict(x_test) # x_test comes from your dataset (in this case MNIST)
decoded_imgs = decoder.predict(encoded_imgs)

## CBIR overview

![](Block-diagram-of-CBIR-system.png)